# Swarm_Dst 科研手册

## 0. 本机的conda环境选择
Swarm_Dst

pytorch



## 1. 数据读取操作

    对下载到的SWarm数据与Dst数据进行预处理，Swarm数据下载方法见readme.ipynb，此次下载的Swarm数据中存在有卫星提供的Dst值，
    同时，也有从地磁台站下载解算得到的Dst值，两者存在差异，此次实验将会对两种Dst数值均进行实验解算。

Swarm_Dst_finished脚本功能：
读取Swarm卫星数据，并进行数据筛选，筛选出特定经纬度范围内，以及特定时间段内的数据
1. 经纬度手动调整，参考依据：文献Fast Dst computation by applying deep learning to Swarm satellite magnetic data
2. 特定时间段，最安静的10天+最受干扰的5天，参考依据：文献Fast Dst computation by applying deep learning to Swarm satellite magnetic data + 
   从世界地磁数据中心下载的QDdays数据，格式见网站。
3. 输出数据文件保存至Swarm_select.npy，目前仅有SWarm提供的Dst数值，并未加入台站测量的Dst值  !!!结合有关产品手册来看两者的dst值应该是一样的
4. ['Spacecraft', 'Timestamp', 'Latitude', 'Longitude', 'Radius', 'B_NEC', 'B_NEC_CHAOS-internal', 'Dst', 'QDLat', 'QDLon'] !!!!getdata.py下载得到的文件中每一行的参数顺序不一样 切记
5. !!! 环境选用Swarm_Dst

In [ ]:
globals().clear()
import cdflib
import os
import numpy as np
import myfunction
import logging


#----------------------确定筛选参数
belt_width_lat = 0.2
lat_cen        = [-40,-30,-20,20,30,40]  # 单位：度

belt_width_lon = 360.0
lon_cen        = [0]

Swarm_dir      = "../Swarm_Data/"
Dst_index      = "../Dst_index/index.dat"
QDday          = "../QDday/QDday.txt"
select_dir     = "../Swarm_select/"

#----------------------读取QDdays文件并保存到相应数据矩阵中

QDday_data = myfunction.QDread(QDday)
QDday_data = np.delete(QDday_data,range(0,11),axis=0)  # 将QDday数据的时间轴与Swarm数据对齐

#--------------------- Create a output logging
logger = logging.getLogger('mylogger')
logger.setLevel(logging.DEBUG)
logger_name = 'my_log.txt'
fh = logging.FileHandler(logger_name)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

#-----------------------创建保存输出文件的文件夹
if not os.path.exists(select_dir):
    os.makedirs(select_dir)


#----------------------获取所有数据的文件名
files = sorted(os.listdir(Swarm_dir))   # sorted函数用于排序


#------------------------文件戳 记录第几个文件
file_epoch = 1

for file_name in files:  
    file_path = Swarm_dir + file_name
    cdf_file = cdflib.CDF(file_path)
    var = cdf_file.cdf_info()
    #----------------------针对每一个文件中的数据进行筛选 
    data_mat_finish = np.ones((1,14))   # 初始化数据保存矩阵，对数据文件中的每一行进行筛选判断，若满足条件则加入其中 第一行是无意义的1，后续需要删除

    #-----------------------------

    raw_data = cdf_file.varget(var.zVariables[0])   #读入数据文件中的所有初始数据
    for i in ['Timestamp', 'Latitude', 'Longitude', 'Radius', 'B_NEC', 'B_NEC_CHAOS-internal', 'Dst', 'QDLat', 'QDLon']:
        data0 = cdf_file.varget(i)
        raw_data = np.column_stack((raw_data,data0))

    for i in range(0,raw_data.shape[0]):           # 进行数据筛选
        raw_data_test   = raw_data[i,:].reshape(1,-1)
        lat             = float(raw_data_test[0,2])
        lon             = float(raw_data_test[0,3])
        time_           = cdflib.cdfepoch.to_datetime(float(raw_data_test[0,1]))
        year            = int(str(time_[0])[0:4])
        mon             = int(str(time_[0])[5:7])
        day             = int(str(time_[0])[8:10])


        QDday_data_test = QDday_data[file_epoch-1,:].reshape(1,-1)
        QD_year         = QDday_data_test[0,0]
        QD_mon          = QDday_data_test[0,1]
        
        if (year != QD_year) or (mon != QD_mon) :   # 判断时间轴是否对齐，否则跳出循环
            print(f"{year}-{mon}Time is wrong!!!")
            break
        
        if not (day in QDday_data_test[0,2:17]):
            continue
        elif not myfunction.location_pd(lat,belt_width_lat,lat_cen):
            continue
        elif not myfunction.location_pd(lon,belt_width_lon,lon_cen):
            continue
        
        data_mat_finish = np.row_stack((data_mat_finish,raw_data_test))

    log_message    =  file_name+"has been calculated."
    logger.debug(log_message)
    #-----------------------------------------------
    file_epoch += 1
        
    #---------------------------------------------------save data
    data_mat_finish = np.delete(data_mat_finish,0,axis=0)   # 第一行需要删除
    save_file       = select_dir + 'Swarm_data_' + str(year).zfill(4) + str(mon).zfill(2) + '_finishedtest.npy'
    np.save(save_file,data_mat_finish)     


## 2. 数据预处理
    从卫星观测值中扣除地磁场内部贡献（地核+地壳）


Swarm_Dst_process脚本功能：
读取筛选后的Swarm卫星数据（/Swarm_select），并进行数据预处理，扣除地磁场内部影响，并输出后续深度学习所需要的数据格式,保存至"train_valid_database/"文件夹
1. 选择以下7个变量作为网络的输入“特征”：地磁纬度、地磁经度、地磁当地时间(MLat, MLon and MLT)，卫星高度以及三个地磁分量残差(res. X, Y和Z)。参考依据：文献Fast Dst computation 
   by applying deep learning to Swarm satellite magnetic data
2. 筛选出训练集、验证集，本次实验的思路是：使用交叉验证法，分成10组，同时由于Dst的特性，有实时Dst值，临时Dst值和最终Dst值（三者的计算方法和详细定义见Version definition Dst）
   尽量保证三种Dst值的训练集和验证集分布相同   ！！！由于Dst会定期更新，截止本次实验，最终Dst值（~2016-12）、临时Dst值（2017-01~2023-06）、实时Dst值（2023-07~）
3. 输出数据文件夹"train_valid_database/"
4. ['Spacecraft', 'Timestamp', 'Latitude', 'Longitude', 'Radius', 'B_NEC', 'B_NEC_CHAOS-internal', 'Dst', 'QDLat', 'QDLon']  !!!!getdata.py下载得到的文件中每一行的参数顺序不一样 切记  在第一步中需要进行调整
5. !!! 环境选用Swarm_Dst

In [ ]:
globals().clear()
import os
import cdflib
import numpy as np
import myfunction
import logging
import random
import math


#----------------------确定输入参数
select_dir             = "../Swarm_select/"  # 上一步的输出文件夹
groups_num             = 10     # 交叉验证法的总组数

final_Dst_begin        = [2013,12,1,0,0,0,0]   # 数字含义：年。月，日，时，分，秒，毫秒
final_Dst_end          = [2016,12,31,23,59,59,999]
provisional_Dst_begin  = [2017,1,1,0,0,0,0]   # 数字含义：年。月，日，时，分，秒，毫秒
provisional_Dst_end    = [2023,6,30,23,59,59,999]
real_time_Dst_begin    = [2023,7,1,0,0,0,0]   # 数字含义：年。月，日，时，分，秒，毫秒
real_time_Dst_end      = [2023,12,31,23,59,59,999]

train_valid_data_dir   = "../train_valid_database/"

#--------------------- Create a output logging
logger = logging.getLogger('mylogger')
logger.setLevel(logging.DEBUG)
logger_name = 'my_log.txt'
fh = logging.FileHandler(logger_name)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

#-----------------------创建保存输出文件的文件夹
if not os.path.exists(train_valid_data_dir):
    os.makedirs(train_valid_data_dir)

#----------------------获取所有数据的文件名
files = sorted(os.listdir(select_dir))   # sorted函数用于排序

#----------------------读取筛选得到Swarm_select数据，进行汇总与分类.

final_mat          = np.zeros((1,14))    # 初始化数据保存矩阵，对数据文件中的每一行进行筛选判断，若满足条件则加入其中 第一行是无意义的0，后续需要删除
provisional_mat    = np.zeros((1,14))
real_time_mat      = np.zeros((1,14))


for file_name in files:
    data_matrix = np.load(select_dir+file_name)
    time_epoch  = data_matrix[0,1]
    if (float(time_epoch) >= cdflib.cdfepoch.compute_epoch(final_Dst_begin)) and (float(time_epoch) <= cdflib.cdfepoch.compute_epoch(final_Dst_end)) :
        final_mat           =  np.vstack((final_mat,data_matrix))
        continue
    elif (float(time_epoch) >= cdflib.cdfepoch.compute_epoch(provisional_Dst_begin)) and (float(time_epoch) <= cdflib.cdfepoch.compute_epoch(provisional_Dst_end)) :
        provisional_mat     =  np.vstack((provisional_mat,data_matrix))
        continue
    elif (float(time_epoch) >= cdflib.cdfepoch.compute_epoch(real_time_Dst_begin)) and (float(time_epoch) <= cdflib.cdfepoch.compute_epoch(real_time_Dst_end)) :
        real_time_mat       =  np.vstack((real_time_mat,data_matrix))
        continue
    
    log_message    =  file_name + " has been calculated."
    logger.debug(log_message)


#---------------------------------------------------save data 分别保存到对应的三类Dst数值中
final_mat        = np.delete(final_mat,0,axis=0)   # 第一行需要删除
provisional_mat  = np.delete(provisional_mat,0,axis=0)
real_time_mat    = np.delete(real_time_mat,0,axis=0)

save_file        = train_valid_data_dir + 'final_mat.npy'
np.save(save_file,final_mat)    
save_file        = train_valid_data_dir + 'provisional_mat.npy'
np.save(save_file,provisional_mat)
save_file        = train_valid_data_dir + 'real_time_mat.npy'
np.save(save_file,real_time_mat) 

#--------------------------------------------------对三类Dst数值进行分组，打乱顺序后，分为n组，进行交叉验证
#--------------------------------------------------在pytorch中可以进行乱序加载，由于在编写此处代码时，尚未知道这个功能，因此额外写了乱序

final_index              =  list(range(0,final_mat.shape[0]))
random.shuffle(final_index)
final_mat[:]             =  final_mat[final_index]

provisional_index        =  list(range(0,provisional_mat.shape[0]))
random.shuffle(final_index)
provisional_mat[:]       =  provisional_mat[provisional_index]

real_time_index          =  list(range(0,real_time_mat.shape[0]))
random.shuffle(final_index)
real_time_mat[:]         =  real_time_mat[real_time_index]

final_index_start        = 0         #由于矩阵的行数不一定被n组整除，因此奇数组是向下取整，偶数组向上取整
provisional_index_start  = 0
real_time_index_start    = 0
for i in range(1,(groups_num+1)) :

    save_file      = train_valid_data_dir + 'database' + str(i).zfill(2) + '.npy'

    if (i % 2 == 1) and (i != groups_num):
        database   =  final_mat[final_index_start:(final_index_start + int(final_mat.shape[0]/groups_num)),:]
        database   =  np.vstack((database,provisional_mat[provisional_index_start:(provisional_index_start + int(provisional_mat.shape[0]/groups_num)),:]))
        database   =  np.vstack((database,real_time_mat[real_time_index_start:(real_time_index_start + int(real_time_mat.shape[0]/groups_num)),:]))
        final_index_start        += int(final_mat.shape[0]/groups_num)
        provisional_index_start  += int(provisional_mat.shape[0]/groups_num)
        real_time_index_start    += int(real_time_mat.shape[0]/groups_num)
    elif (i % 2 == 0) and (i != groups_num):
        database   =  final_mat[final_index_start:(final_index_start + math.ceil(final_mat.shape[0]/groups_num)),:]
        database   =  np.vstack((database,provisional_mat[provisional_index_start:(provisional_index_start + math.ceil(provisional_mat.shape[0]/groups_num)),:]))
        database   =  np.vstack((database,real_time_mat[real_time_index_start:(real_time_index_start + math.ceil(real_time_mat.shape[0]/groups_num)),:]))
        final_index_start        += math.ceil(final_mat.shape[0]/groups_num)
        provisional_index_start  += math.ceil(provisional_mat.shape[0]/groups_num)
        real_time_index_start    += math.ceil(real_time_mat.shape[0]/groups_num)
    else :
        database   =  final_mat[final_index_start:,:]
        database   =  np.vstack((database,provisional_mat[provisional_index_start:,:]))
        database   =  np.vstack((database,real_time_mat[real_time_index_start:,:]))
    
    np.save(save_file,database)
    log_message    =  'database' + str(i).zfill(2) + " has been calculated."
    logger.debug(log_message)







## 3. 深度学习框架的搭建

    此步骤环境选用pytorch

### 回归模型架构
    创建一个MLP回归模型的类

In [ ]:
import torch
from torch.nn import Linear, ReLU, ModuleList, Sequential, Dropout, Softmax, Tanh, Sigmoid

#-------------------------对类进行继承和重新定义
class MLP(torch.nn.Module) :
    def __init__(self, input_n, output_n, num_layer, layer_list, dropout=0.5) :
        super(MLP, self).__init__()
        self.input_n    = input_n
        self.outout_n   = output_n
        self.num_layer  = num_layer
        self.layer_lsit = layer_list

        self.input_layer = Sequential(
            Linear(input_n, layer_list[0], bias=False),
            Sigmoid()
        )
        self.hidden_layer = Sequential()

        for index in range(num_layer-1) :
            self.hidden_layer.extend([Linear(layer_list[index], layer_list[index+1], bias=False), Sigmoid()])

        self.dropout = Dropout(dropout)

        self.output_layer = Sequential(
            Linear(layer_list[-1], output_n, bias=False)
            # ReLU()
            # Softmax(dim=1)
        )

    #-------------------此处forward函数强烈建议命名为forward，由于魔术方法的存在，会自动调用forward方法，
    #-------------------即object.forward(x) 的作用等于 object(x) 但不建议写object.forward(x)
    #-------------------因为由于魔术方法的存在，这样写会导致这个方法调用两次，详细机理见收藏的链接
    def forward(self, x) :                       
        in_put = self.input_layer(x)
        hidden = self.hidden_layer(in_put)
        hidden = self.dropout(hidden)
        output = self.output_layer(hidden)
        output = output.view(-1)
        return output


### 深度学习相关函数封装

In [ ]:
import torch
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import time
from datetime import timedelta
import os


#-------------------------定义一个数据格式的类，继承自torch.utils.data.Dataset
class Dataset_(torch.utils.data.Dataset) :
    def __init__(self, data_df) :   #-----------此处由于父类并没有init，因此不必使用super关键字
        self.label   = torch.from_numpy(data_df['Dst'].values).to(torch.float)
        self.data    = torch.from_numpy(data_df[data_df.columns[:-1]].values).to(torch.float)

    # --------复写父类的__getitem__
    def __getitem__(self, index) :
        batch_data  = self.get_batch_data(index)
        batch_label = self.get_batch_label(index)
        return   batch_data, batch_label        # 此处必须严格与后面for index,(trains, labels) in enumerate(dataloader)中的括号内容对应
                                                # 先特征值，后标签值

    def classes(self) :
        return self.label
    
    def __len__(self) :
        return self.data.size(0)
    
    def get_batch_label(self,index) :
        return self.label[index]
    
    def get_batch_data(self, index) :
        return self.data[index]


#----------------------------保存数据，加载数据
class Config_finished() : #-----------------已经划分好训练集，验证集，测试集
    '''
    针对本次实验，由于在之前的步骤中，已将数据分为10组
    此次可直接用于封装已经划分好训练集，验证集，测试集的数据
    '''
    def __init__(self, data_dir_path, name, train_list, valid_list, test_list, batch_size, learning_rate, epoch) -> None:
        """
        data_dir_path     : string 数据文件所在文件夹路径
        name              : string 模型名字
        train_list        : lsit 训练集的编号  7个
        valid_list        : list 验证集的编号  1个
        test_list         : list 测试集合的编号  2个
        batch_size        : int 多少条数据组成一个batch
        learning_rate     : float 学习率
        epoch             : int 输入学习轮数，在输入模型训练后，输出为实际训练的轮数
        train_loader, valid_loader, test_loader  : 训练数据、验证数据、测试数据
        """

        self.name           = name
        self.data_dir_path  = data_dir_path
        self.train_list     = train_list
        self.valid_list     = valid_list
        self.test_list      = test_list
        self.batch_size     = batch_size
        self.learning_rate  = learning_rate
        self.epoch          = epoch
        self.train_loader, self.valid_loader, self.test_loader = self.load_tdt()

    #------------------将读取的numpy文件转化为dataframe
    def transform(self, path_name) :
        data_npy   = np.load(path_name)
        data_npy   = np.delete(data_npy, 0, axis=1)  # 删除了第一行 也就是Spacecraft
        data_npy   = data_npy.astype(np.float32)
        df         = pd.DataFrame(data_npy)
        Config_finished.if_nan(df)
        df.columns = ['Timestamp', 'Latitude', 'Longitude', 'Radius', 'B_N', 'B_E', 'B_C',
                    'B_N_CHAOS-internal', 'B_E_CHAOS-internal', 'B_C_CHAOS-internal', 'Dst', 'QDLat', 'QDLon']
        df['B_N_minus'] = df['B_N'] - df['B_N_CHAOS-internal']
        df['B_E_minus'] = df['B_E'] - df['B_E_CHAOS-internal']
        df['B_C_minus'] = df['B_C'] - df['B_C_CHAOS-internal']
        df.drop(['Timestamp','B_N','B_E','B_C','B_N_CHAOS-internal','B_E_CHAOS-internal','B_C_CHAOS-internal',
                 'Radius','Latitude', 'Longitude', 'QDLat', 'QDLon'], axis=1, inplace=True)
        df         = df.reindex(columns=['B_N_minus', 'B_E_minus', 'B_C_minus', 'Dst'])
        
        return df
    

    #------------------生成数据集集并封装成Dataloader类，需要读入数据集的选项
    def dataset_create(self, list) :
        '''
        list输入你想要生成总数据集的子数据集编号，在此次实验中是1-10
        '''
        i =  list[0]
        path_name           = self.data_dir_path + "database" + str(i).zfill(2) + ".npy"
        data_frame_finished = self.transform(path_name) 
        for i in list[1:] :
            path_name           = self.data_dir_path + "database" + str(i).zfill(2) + ".npy"
            data_frame          = self.transform(path_name) 
            data_frame_finished = pd.concat([data_frame_finished, data_frame])

        dataset  =  Dataset_(data_frame_finished)
        return torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
    
    #-------------------生成训练集、验证集、测试集
    def load_tdt(self) :
        train_loader = self.dataset_create(self.train_list)
        valid_loader = self.dataset_create(self.valid_list)
        test_loader  = self.dataset_create(self.test_list)
        return train_loader, valid_loader, test_loader

    #-------------------判断数据集是否有空数值 (输入数据格式为pd.dataframe) 
    #-------------------该方法为静态方法，可以不用通过实例化来使用
    @staticmethod
    def if_nan(dataframe) :
        if dataframe.isnull().any().any():
            emp = dataframe.isnull().any()
            print(emp[emp].index)
            print("Empty data exists")
            sys.exit(0) #---------程序正常退出，并进行变量清理等等



#--------------------------------回归模型的训练、测试和评估
class REG_model() :
    '''
    针对本次实验，对实验数据进行训练、测试和评估等等
    并可直接进行可视化操作
    '''

    def __init__(self, model, config) -> None:
        self.model = model
        self.config = config


    def run(self) :
        self.train_(self.model)

    def train_(self, model) :
        dev_best_loss = float('inf')
        strat_time = time.time()
        #-------------------------将模型切换为训练模型
        model.train()
        #------------------------定义优化器
        optimizer = torch.optim.Adam(model.parameters(), lr=self.config.learning_rate)
        acc_list = [[], []]
        loss_list = [[], []]
        #-------------------------记录损失不下降的epoch数，到达20之后就直接退出 => 训练无效，再训练下去可能过拟合
        break_epoch = 0

        for epoch in range(self.config.epoch) :
            print('Epoch [{}/{}]'.format(epoch+1,self.config.epoch))
            for index, (trains, labels) in enumerate(self.config.train_loader) :
                # 归零
                model.zero_grad()      #---------------进行梯度归零
                # 得到预测结果，进行正向解算
                outputs = model(trains)
                # 计算MSELOSS函数
                loss_      = torch.nn.MSELoss() #-----------实例化一个对象
                loss_mean  = loss_(outputs, labels)
                # 反向传播loss
                loss_mean.backward()
                # 参数优化与参数更新
                optimizer.step()
                # 每迭代100次或跑完一个epoch，进行一次验证
                if (index % 100 == 0 and index != 0) or index == (len(self.config.train_loader) - 1) :
                    true = labels.detach().cpu().numpy()
                    # 预测数据
                    predict = outputs.detach().cpu().numpy()
                    # 计算训练集的准确度 决定系数R2   基于sklearn库 需要转化成nump格式
                    # 注意torch库有自带的计算决定系数的函数，输入值与sklearn库有所不同，需要注意
                    train_acc = r2_score(true, predict)
                    # 计算验证集的准确度 决定系数R2    注意事项同上
                    [dev_acc, dev_loss, dev_mse] = self.evaluate(model)
                    # 验证loss函数是否进步
                    if dev_loss < dev_best_loss :
                        dev_best_loss = dev_loss
                        improve = '*'
                        break_epoch = 0
                    else :
                        improve = ''
                        break_epoch += 1
                    # 计算消耗时间
                    time_dif = self.get_time_dif(start_time=strat_time)

                    # 输出阶段性成果 .item() 方法表示的是  将单元素tensor量 转化为float
                    msg = 'Iter:{0:>6},  Train Loss: {1:>5.3},  Train R2: {2:>6.3},  Val Loss: {3:>5.3},  Val R2: {4:>6.3},  Val Mse: {5:>6.3},  Time: {6} {7}'
                    print(msg.format(index, loss_mean.item(), train_acc, dev_loss, dev_acc, dev_mse, time_dif, improve))
                    # 每当跑完一个epoch，记录画图数据
                    if index == (len(self.config.train_loader) - 1) :
                        acc_list[0].append(train_acc)
                        acc_list[1].append(dev_acc)
                        loss_list[0].append(loss_mean.item())
                        loss_list[1].append(dev_loss)

                    # 转化为训练模式
                    model.train()
            # 设定早退，防止过拟合，如果20次验证，损失函数没有减小，直接退出训练
            if break_epoch > 20 :
                self.config.epoch = epoch + 1
                break
        # 测试
        self.test(model)
        # 画图 图片默认的保存地址是src文件夹上一级的images文件夹
        self.draw_curve(acc_list, loss_list, self.config.epoch)

                    
    def test(self, model) :
        start_time = time.time()
        # 测试集准确度R2，损失函数值，MSE
        [test_acc, test_loss, test_mse] = self.evaluate(model, test=True)
        msg = 'Test R2: {0:>5.3},  Test loss: {1:>6.3},  Test MSE: {2:>6.3}'
        print(msg.format(test_acc, test_loss, test_mse))
        time_dif = self.get_time_dif(start_time=start_time)
        print("Time usage:", time_dif)

    
    def evaluate(self, model, test=False) :
        '''
        test=False 使用验证集
        test=True  使用测试集
        '''
        # 转变模型模式
        model.eval()
        loss_total  = 0
        predict_all = np.array([], dtype=float)
        labels_all  = np.array([], dtype=float)

        if test :
            with torch.no_grad() :
                for index, (valids, labels) in enumerate(self.config.test_loader) :
                    outputs     = model(valids)
                    loss_       = torch.nn.MSELoss() #-----------实例化一个对象
                    loss_mean   = loss_(outputs, labels)
                    loss_total  += loss_mean
                    labels      = labels.detach().cpu().numpy()
                    predict     = outputs.detach().cpu().numpy()
                    labels_all  = np.append(labels_all, labels)
                    predict_all = np.append(predict_all, predict)

        else :
            with torch.no_grad() :
                for index, (valids, labels) in enumerate(self.config.valid_loader) :
                    outputs     = model(valids)
                    loss_       = torch.nn.MSELoss() #-----------实例化一个对象
                    loss_mean   = loss_(outputs, labels)
                    loss_total  += loss_mean
                    labels      = labels.detach().cpu().numpy()
                    predict     = outputs.detach().cpu().numpy()
                    labels_all  = np.append(labels_all, labels)
                    predict_all = np.append(predict_all, predict)

        dev_acc = r2_score(labels_all, predict_all)
        dev_mse = mean_squared_error(labels_all, predict_all)
        #-----------------注意：loss_total / len(self.config.test_loader) ->>>>   dev_loss
        #-----------------表示的是损失函数和的均值 
        if test :
            return dev_acc, loss_total / len(self.config.test_loader), dev_mse
        else :
            return dev_acc, loss_total / len(self.config.valid_loader), dev_mse


    # 计算时间损耗
    def get_time_dif(self, start_time) :
        end_time = time.time()
        time_dif = end_time - start_time
        return timedelta(seconds=int(round(time_dif)))
    
    # 可视化输出
    def draw_curve(self, acc_list, loss_list, epochs) :
        #-----------------------创建保存输出文件的文件夹
        if not os.path.exists('../images/'):
            os.makedirs('../images/')
        
        x = range(0, epochs)
        y1 = loss_list[0]
        y2 = loss_list[1]
        y3 = acc_list[0]
        y4 = acc_list[1]
        plt.figure(figsize=(13, 13))
        plt.subplot(2, 1, 1)
        plt.plot(x, y1, color="blue", label="train_loss", linewidth=2)
        plt.plot(x, y2, color="orange", label="val_loss", linewidth=2)
        plt.title("Loss_curve", fontsize=20)
        plt.xlabel(xlabel="Epochs", fontsize=15)
        plt.ylabel(ylabel="Loss", fontsize=15)
        plt.legend()
        plt.subplot(2, 1, 2)
        plt.plot(x, y3, color="blue", label="train_acc", linewidth=2)
        plt.plot(x, y4, color="orange", label="val_acc", linewidth=2)
        plt.title("Acc_curve", fontsize=20)
        plt.xlabel(xlabel="Epochs", fontsize=15)
        plt.ylabel(ylabel="Accuracy", fontsize=15)
        plt.legend()
        plt.savefig("../images/"+self.config.name+"_Loss&acc.png")



In [ ]:
import MLP_reg
import my_utils
import gc

config_ = my_utils.Config_finished(data_dir_path='../train_valid_database/',
                             name='regress',
                             train_list=[1,2,3,4,5,6,7],
                             valid_list=[8,9],
                             test_list=[10],
                             batch_size=1000,
                             learning_rate=0.01,
                             epoch=20)

rag = MLP_reg.MLP(        
        input_n=3,
        output_n=1,
        num_layer=3,
        layer_list=[7, 14, 7],
        dropout=0.0)


model = my_utils.REG_model(rag, config_)
model.run()
gc.collect()

In [ ]:
import my_utils





a = my_utils.Config_finished(data_dir_path='../train_valid_database/',
                             name='reg',
                             train_list=[1,2,3,4,5,6,7],
                             valid_list=[8],
                             test_list=[9,10],
                             batch_size=10,
                             learning_rate=0.1,
                             epoch=10)


rag = MLP_reg.MLP(        
        input_n=13,
        output_n=1,
        num_layer=3,
        layer_list=[9, 29, 9],
        dropout=0.5)

for index,trains, labels in enumerate(a.valid_loader):
    print(index)
    # print(trains)
    print(labels)
    print()

以下为测试脚本。不可放进主程序运行！！！！！！

In [ ]:
globals().clear()
import MLP_reg
import torch
import torch.nn
import torch.utils
import torch.utils.data

rag = MLP_reg.MLP(        
        input_n=5,
        output_n=1,
        num_layer=4,
        layer_list=[512, 128, 32, 8],
        dropout=0.5)
x=torch.tensor([1.0,2.0,3.0,4.0,7.0])
#x=torch.tensor([[1.0,2.0,3.0,4.0,7.0],[1.0,2.0,3.0,4.0,6.0]])
print(x[-1:])
print(x[:-1])

a=[1,2,3]

print(type(rag.parameters()))



In [ ]:
globals().clear()
import torch

x=torch.tensor([1.0,2.0,3.0,4.0,7.0])
y=torch.tensor([1.0,2.0,3.0,4.0,7.0])
z=torch.tensor([1.0,2.0,3.0,4.0,7.0],requires_grad=True)
m=x+y+z
print(m.requires_grad)
with torch.no_grad() :
    q = m+y+z
    print(torch.no_grad().prev)
    print(x.requires_grad)
    print(m.requires_grad)

print(x.requires_grad)
print(m.requires_grad)

以下为一个简单线性回归的过程实例

In [ ]:
globals().clear()
import numpy as np
import matplotlib.pyplot as plt
import torch
import random

def synthetic_data(w,b,num_examples) :
    X = torch.normal(0,1,(num_examples,len(w)))
    y = torch.matmul(X,w) + b
    y += torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1))


def data_iter(batch_size, features, labels) :
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size) :
        batch_indices = torch.tensor(indices[i:min(i + batch_size,num_examples)])
        yield features[batch_indices], labels[batch_indices]


def linreg(X, w, b) :
    return torch.matmul(X,w) + b

def squared_loss(y_hat, y) :
    return (y_hat - y.reshape(y_hat.shape)) **2 /2 


def sgd(params, lr, batch_size) :
    with torch.no_grad() :
        for param in params :
            param -= lr * param.grad / batch_size
            param.grad.zero_()


true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w,true_b,1000)

batch_size = 10
for x, y in data_iter(batch_size, features, labels) :
    print(x , '\n' , y)
    break


w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

lr = 0.03
num_epochs = 3

for epoch in range(num_epochs) :
    for X, y in data_iter(batch_size, features, labels) :
        l = squared_loss(linreg(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)

    with torch.no_grad() :
        train_l = squared_loss(linreg(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')
